# Kaufabsicht vorhersagen

Der folgende Code vervollständigt das Beispiel aus dem Abschnitt [Daten vorbereiten](preprocessing.ipynb), in dem die Kaufabsicht von Klient\*innen vorhergesagt werden soll.

## Bibliotheken installieren

Führen Sie die folgende Zelle aus, um zu überprüfen, dass alle notwendigen Bibliotheken installiert sind.

In [128]:
!pip install scikit-learn
!pip install numpy
!pip install pandas
!pip install matplotlib

## Bibliotheken importieren

Die folgenden Bibliotheken brauchen Sie für die meisten Aufgaben. Ihr Import steht daher meistens am Anfang Ihrer Programme oder Notebooks.

In [129]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Daten importieren

Die Daten, die in diesem Beispiel als CSV-Datei vorliegen, wird mithilfe von *pandas* importiert.

In [130]:
df = pd.read_csv("data/dataset.csv")

Das geladene Datenset kann hier im Notebook angezeigt werden.

In [131]:
df

,Land,Alter,Gehalt,gekauft
0,Frankreich,44.0,72000.0,Nein
1,Griechenland,27.0,48000.0,Ja
2,Deutschland,30.0,54000.0,Nein
3,Griechenland,38.0,61000.0,Nein
4,Deutschland,40.0,NaN,Ja
5,Frankreich,35.0,58000.0,Ja
6,Griechenland,NaN,52000.0,Nein
7,Frankreich,48.0,79000.0,Ja
8,Deutschland,50.0,83000.0,Nein
9,Frankreich,37.0,67000.0,Ja


## Featurematrix auswählen

Zunächst werden die Features in einer Variable zusammengefasst, die einer **Matrix** entspricht.

In [132]:
X = df.iloc[:, :-1].values

In [133]:
y = df.iloc[:, -1].values

In [134]:
print(X)

[['Frankreich' 44.0 72000.0]
 ['Griechenland' 27.0 48000.0]
 ['Deutschland' 30.0 54000.0]
 ['Griechenland' 38.0 61000.0]
 ['Deutschland' 40.0 nan]
 ['Frankreich' 35.0 58000.0]
 ['Griechenland' nan 52000.0]
 ['Frankreich' 48.0 79000.0]
 ['Deutschland' 50.0 83000.0]
 ['Frankreich' 37.0 67000.0]]


In [135]:
print(y)

['Nein' 'Ja' 'Nein' 'Nein' 'Ja' 'Ja' 'Nein' 'Ja' 'Nein' 'Ja']


## Fehlende Daten berücksichtigen

Wenn wenige Daten fehlen, können diese Datensätze in der Regel ignoriert bzw. entfernt werden. Fehlen in vielen Datensätzen Daten, gibt es verschiedene Strategien, diese zu ersetzen. Eine davon ist, den Mittelwert aus den vorhandenen Daten zu bilden. Hierfür stellt *scikit-learn* ein Verfahren zur Verfügung.

In [136]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 1:3])
X[:, 1:3] = imputer.transform(X[:, 1:3])

In [137]:
print(X)

[['Frankreich' 44.0 72000.0]
 ['Griechenland' 27.0 48000.0]
 ['Deutschland' 30.0 54000.0]
 ['Griechenland' 38.0 61000.0]
 ['Deutschland' 40.0 63777.77777777778]
 ['Frankreich' 35.0 58000.0]
 ['Griechenland' 38.77777777777778 52000.0]
 ['Frankreich' 48.0 79000.0]
 ['Deutschland' 50.0 83000.0]
 ['Frankreich' 37.0 67000.0]]


:::{.callout-tip}
### Aufgabe
Recherchieren Sie verwendeten Klassen und Methoden [in der Dokumentation von *scikit-learn*](https://scikit-learn.org/stable/index.html).

:::

## Kategorien in den Daten kodieren

Damit der Algorithmus die Daten schnell und eindeutig verarbeiten kann, müssen Textwerte als Zahlenwerte kodiert werden, hier also die Länder. Ein gängiges Verfahren ist hierbei, binäre Vektoren aus den möglichen Kombinationen der Kategorien zu erstellen. Bei diesem Vorgehen werden neue Spalten angelegt, die diese Vektoren abbilden. [Lesen Sie hierzu auch den Artikel über **One Hot Encoding**, wie dieses Verfahren genannt wird](https://www.educative.io/blog/one-hot-encoding).

Da im vorliegenden Datenset sowohl in den unabhängigen als auch in der abhängigen Variable kategorische Daten vorliegen, ist der Vorgang zweimal notwendig.

### Die unabhängige Variable kodieren

Auch hierfür werden wieder Methoden von *scikit-learn* verwendet.

In [138]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [139]:
print(ct)

ColumnTransformer(remainder='passthrough',
                  transformers=[('encoder', OneHotEncoder(), [0])])


In [140]:
print(X)

[[0.0 1.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [1.0 0.0 0.0 40.0 63777.77777777778]
 [0.0 1.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 48.0 79000.0]
 [1.0 0.0 0.0 50.0 83000.0]
 [0.0 1.0 0.0 37.0 67000.0]]


### Die abhängige Variable kodieren

Da die Werte der abhängigen Variablen binär abgebildet werden können, ist hier One-Hot-Encoding nicht notwendig. 

In [141]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [142]:
print(y)

[1 0 1 1 0 0 1 0 1 0]


## Daten aufteilen in Training- und Testset

Für den eigentlichen Prozess des "Maschinenlernens" wird das Datenset in ein **Trainingsset** und ein **Testset** aufgeteilt.

Mit dem Trainingsset wird das Machine-Learning-Modell trainiert, mit dem Testset wird die Vorhersagegüte des Modells getestet. Hierbei wird so getan, als ob es sich bei den Testdaten um zukünftige reale Daten handelt.

*scikit-learn* stellt auch hierfür ein Verfahren bereit.

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [144]:
print(X_train)

[[0.0 1.0 0.0 37.0 67000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 48.0 79000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 44.0 72000.0]
 [0.0 1.0 0.0 35.0 58000.0]]


In [145]:
print(X_test)

[[1.0 0.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 40.0 63777.77777777778]]


In [146]:
print(y_train)

[0 0 1 0 1 1 0]


In [147]:
print(y_test)

[1 1 0]


## Feature Scaling

Feature Scaling ist die Anpassung der unabhängigen Daten in einer Weise, dass sie alle im gleichen Verhältnis zueinander stehen. Dadurch wird vermieden, dass ein Feature andere Features dominiert. 

**Wichtig: Feature Scaling wird immer erst nach der Aufteilung des Datensets durchgeführt! Damit wird Informationsverlust bei den Testdaten vermieden.**

In [148]:
X_test[:, 3:]

array([[30.0, 54000.0],
       [50.0, 83000.0],
       [40.0, 63777.77777777778]], dtype=object)

In [149]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc.transform(X_test[:, 3:])

In [150]:
print(X_train)

[[0.0 1.0 0.0 -0.2029809015697542 0.44897082661305115]
 [0.0 0.0 1.0 -1.821689357126023 -1.4170641714974423]
 [0.0 0.0 1.0 0.08478949052913817 -1.0242146982110225]
 [0.0 1.0 0.0 1.5775983995421416 1.62751924647231]
 [0.0 0.0 1.0 -0.041110056014127316 -0.14030338331657835]
 [0.0 1.0 0.0 0.930115017319634 0.9400326682210757]
 [0.0 1.0 0.0 -0.526722592681008 -0.4349404882813931]]


In [151]:
print(X_test)

[[1.0 0.0 0.0 -1.3360768204591424 -0.8277899615678128]
 [1.0 0.0 0.0 1.9013400906533953 2.02036871975873]
 [1.0 0.0 0.0 0.28263163509712647 0.13250875091010228]]


## Trainieren des Modells

In [152]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

## Eine Vorhersage mit neuen Daten machen

Einen neuen Datensatz erstellen:

In [153]:
new_data = [0.0, 0.0, 1.0, 39, 160000]

Den Datensatz zum richtigen Typ wandeln und in die richtige Form bringen:

In [154]:
new_data = np.array(new_data).reshape(1, -1)
new_data

array([[0.0e+00, 0.0e+00, 1.0e+00, 3.9e+01, 1.6e+05]])

Das Segment ausgeben, das skaliert werden soll:

In [155]:
new_data[:, 3:]

array([[3.9e+01, 1.6e+05]])

Die Transformation auf das Segment anwenden und zurückschreiben:

In [156]:
new_data[:, 3:] = sc.transform(new_data[:, 3:])
new_data

array([[0.  , 0.  , 1.  , 0.12, 9.58]])

Die Vorhersage machen:

In [157]:
classifier.predict(new_data)

array([0])

## Die Testdaten vorhersagen

Weil es jetzt mehrere Features gibt, ist das Zeichnen eine mehrdimensionalen Graphen nicht möglich. Daher werden im Folgenden die **Profite** aus den Trainingsdaten mit denen aus dem Testset verglichen. Hierbei kann die Güte des Modells erneut an der Abweichung der vorhergesagten zu den Testwerten gemessen werden.

In [158]:
X_test

array([[1.0, 0.0, 0.0, -1.3360768204591424, -0.8277899615678128],
       [1.0, 0.0, 0.0, 1.9013400906533953, 2.02036871975873],
       [1.0, 0.0, 0.0, 0.28263163509712647, 0.13250875091010228]],
      dtype=object)

In [159]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 1, 0])

Um die vorhergesagten und bekannten Daten nebeneinanderstellen zu können, wird das numpy-Array umgeformt:

In [160]:
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

[[0 1]
 [1 1]
 [0 0]]


## Wahrheitsmatrix (*confusion matrix*) ausgeben

[Die Wikipedia](https://de.wikipedia.org/wiki/Beurteilung_eines_bin%C3%A4ren_Klassifikators#Wahrheitsmatrix:_Richtige_und_falsche_Klassifikationen) erklärt sehr gut, wie eine Wahrheitsmatrix (*confusion matrix*) bei der Gütebewertung des Modells helfen kann.

In [167]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Die Wahrheitsmatrix sieht aus wie folgt:")
print(cm)
# Die Rate der korrekten Vorhersagen
print("Der Accuracy Score beträgt {} %.".format(accuracy_score(y_test, y_pred)*100))

Die Wahrheitsmatrix sieht aus wie folgt:
[[1 0]
 [1 1]]
Der Accuracy Score beträgt 66.66666666666666 %.
